In [20]:
import sys
sys.path.insert(0, '/Users/bobo/Code/hypergol-test/heart_failure_detection')
import os
os.environ['PROJECT_PROFILE'] = "MacBook-Pro-4.local_bobo"
os.environ['PROJECT_LOCATION'] = "/Users/bobo/Code/hypergol-test/heart_failure_detection"
from data_models.example_class import ExampleClass
from data_models.other_example import OtherExample
from data_models.ppg_segment_5_min import PpgSegment5Min
from src.data.dataset import Dataset
from datetime import date
from datetime import datetime
from src.config.config import create_config_template, read_config, write_plainconfig, create_new_experiment
from src.config.env_manager import get_env_config, setup_env_config

In [21]:
setup_env_config()
env_config = get_env_config()

Please change the raw_data_folder in config.ini
Using profile:  MacBook-Pro-4.local_bobo


In [3]:
cfg = read_config(env_config['PARAMETER_LOCATION'])

In [ ]:
python3 -m hypergol.cli.create_data_model ppg_segment_5_min id:int:id age:int pid:int event:int nyha:int start_time:datetime "ppg:List[float]" "x:List[float]" "y:List[float]" "z:List[float]"

In [22]:
root = "/Users/bobo/Code/ftp_data/聯發科PPG Raw Data"
col_name = ['datetime', 'ppg_ts', 'ppg', 'acc_ts', 'x', 'y', 'z']



In [ ]:
id: int, age: int, pid: int, event: int, nyha: int, start_time: datetime, ppg: List[float], x: List[float], y: List[float], z: List[float])

In [23]:
dataset=Dataset(
    dataType=PpgSegment5Min,
    location=env_config['raw_data_folder'],
    project='heart-failure',
    branch=env_config['project_profile'],
    name='raw_ppg_test',
    chunkCount=16,
    repoData=None
)

Dummy repodata was used, data lineage disabled


In [24]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import heartpy as hp

In [25]:
def interpolate_acc(df):
    
    acc = df[['acc_ts', 'x', 'y', 'z']].copy()
    ppg = df[['datetime', 'ppg_ts', 'ppg']].copy()

    acc.dropna(inplace=True)
    ppg.dropna(inplace=True)

    x = np.arange(acc.shape[0])
    y_x = np.array(acc['x'])
    y_y = np.array(acc['y'])
    y_z = np.array(acc['z'])

    xnew = np.arange(0, acc.shape[0], acc.shape[0]/ppg.shape[0])
    f_x = interp1d(x, y_x, bounds_error=False, fill_value="extrapolate")
    f_y = interp1d(x, y_y, bounds_error=False, fill_value="extrapolate")
    f_z = interp1d(x, y_z, bounds_error=False, fill_value="extrapolate")

    x_xnew, x_ynew, x_znew = f_x(xnew), f_y(xnew), f_z(xnew)

    print(x.shape, xnew.shape)
    print(y_x.shape, x_xnew.shape)
    df['x_new'], df['y_new'], df['z_new'] = [x_xnew, x_ynew, x_znew]
    return df

def Pythagorean_3d(arr):
    """
    ab = Pythagorean_3d(ppg_df[['x_new', 'y_new', 'z_new']].to_numpy())
    ppg_df['tri_acc'] = np.array(ab)
    """
    a,b,c = np.hsplit(arr, 3)
    return np.around(((a**2 + b**2 + c**2) ** (1/2)), 5)

def df_preprocess(fpath: str):
    
    ppg_df = pd.read_csv(fpath, names=col_name, header=None)

    #  data preprocess
    try:
        # timestsamp preprocessing
        ppg_df['datetime'] = ppg_df['datetime'].replace(
            to_replace=0, method='bfill')
        ppg_df = ppg_df[ppg_df['datetime'] > 0]
        ppg_df['time'] = pd.to_datetime(ppg_df['datetime'], format=TIME_FORMAT)
        ppg_df.index = pd.to_datetime(ppg_df['datetime'], format=TIME_FORMAT)

        # interpolate
        ppg_df = interpolate_acc(ppg_df)
        ppg_df = ppg_df.dropna()

        # get trixial
        ab = Pythagorean_3d(ppg_df[['x_new', 'y_new', 'z_new']].to_numpy())
        ppg_df['tri_acc'] = np.array(ab)
        return ppg_df

    except Exception as e:
        print(e)
        raise RuntimeError

In [26]:
label_df = pd.read_csv("/Users/bobo/Code/ftp_data/patient_data.csv", index_col=0)
fpath = "/Users/bobo/Code/ftp_data/聯發科PPG Raw Data/TVGH002/20190408184835_raw.csv"
TIME_FORMAT = '%Y%m%d%H%M%S'
df = df_preprocess(fpath)

patient_folders = next(os.walk('/Users/bobo/Code/ftp_data/聯發科PPG Raw Data'), (None, None, []))[1]
patient_id_list = sorted([int(f[4:]) for f in patient_folders if (
    f.find('TVGH') != -1 and not f.endswith("(x)"))])
patient_id_list

(4202056,) (4304896,)
(4202056,) (4304896,)


[2,
 3,
 5,
 6,
 7,
 9,
 12,
 13,
 14,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 32,
 33,
 34,
 37,
 38,
 39,
 40,
 41,
 42,
 44,
 45,
 46,
 47,
 48,
 49,
 50]

In [10]:
import logging

FORMAT = '%(asctime)s  %(levelname)s  %(message)s'
logging.basicConfig(
    format=FORMAT, filename="/Users/bobo/Code/hypergol-test/heart_failure_detection/log.txt", level=0)
logging.info("test")

In [ ]:
dataset.delete()

counter = 0

with dataset.open('w') as datasetWriter:
    for pid in patient_id_list:
        logging.info(f"start pid: {pid}")
        patient_num = 'TVGH%03d' % pid
        root = os.path.join('/Users/bobo/Code/ftp_data/聯發科PPG Raw Data', patient_num)
        filenames = next(os.walk(root), (None, None, []))[2]
        filenames = [f for f in filenames if f.find('.csv') != -1]
        attr = label_df.loc[pid][['patient_id', 'csv num', 'Age ', 'Gender', 'NYHA', 'Event label']].to_list()
        for f in filenames:
            logging.info(f"start file: {f}")
            fpath = os.path.join(root, f)
            try:
                ppg_df = df_preprocess(fpath)
            except Exception as e:
                logging.error(f"PID={pid}, id : {counter}, file: {f}, status: {'preprocess error'}: {e}", )
                continue
            window = hp.peakdetection.make_windows(ppg_df['ppg'], 64, windowsize=5*60, overlap=0, min_size=19200)
            for w_start, w_end in window:
                try:
                    sample = ppg_df.iloc[w_start:w_end]['ppg'].to_list()[:19200]
                    acc = ppg_df.iloc[w_start:w_end]['tri_acc'].to_list()[:19200]
                    start = ppg_df.iloc[w_start].time
                    datasetWriter.append(
                        PpgSegment5Min(
                            id = counter,
                            age = attr[2], 
                            pid = attr[0],
                            event = attr[5],
                            nyha = attr[4],
                            start_time = start,
                            ppg = sample, 
                            tri_acc = acc)
                    )
                    counter += 1
                except Exception as e:
                    logging.error(f"PID={pid}, id : {counter}, file: {f}, status: {'writing error'}: {e}", )
                    continue

### test dataset

In [27]:
datasetWriter = dataset.open('w')


for pid in patient_id_list:
    logging.info(f"start pid: {pid}")
    patient_num = 'TVGH%03d' % pid
    root = os.path.join('/Users/bobo/Code/ftp_data/聯發科PPG Raw Data', patient_num)
    filenames = next(os.walk(root), (None, None, []))[2]
    filenames = [f for f in filenames if f.find('.csv') != -1]
    attr = label_df.loc[pid][['patient_id', 'csv num', 'Age ', 'Gender', 'NYHA', 'Event label']].to_list()
    for f in filenames:
        logging.info(f"start file: {f}")
        fpath = os.path.join(root, f)
        try:
            ppg_df = df_preprocess(fpath)
        except Exception as e:
            logging.error(f"PID={pid}, id : {counter}, file: {f}, status: {'preprocess error'}: {e}", )
            continue
        window = hp.peakdetection.make_windows(ppg_df['ppg'], 64, windowsize=5*60, overlap=0, min_size=19200)
        for w_start, w_end in window:
            try:
                sample = ppg_df.iloc[w_start:w_end]['ppg'].to_list()[:19200]
                acc = ppg_df.iloc[w_start:w_end]['tri_acc'].to_list()[:19200]
                start = ppg_df.iloc[w_start].time
                datasetWriter.append(
                    PpgSegment5Min(
                        id = counter,
                        age = attr[2], 
                        pid = attr[0],
                        event = attr[5],
                        nyha = attr[4],
                        start_time = start,
                        ppg = sample, 
                        tri_acc = acc)
                )
                counter += 1
            except Exception as e:
                logging.error(f"PID={pid}, id : {counter}, file: {f}, status: {'writing error'}: {e}", )
                continue
        break
    break
datasetWriter.close()

(5340745,) (5357248,)
(5340745,) (5357248,)


In [12]:
with dataset.open('r') as datasetReader:
    for data in datasetReader:
        test = (data)


## read data

In [29]:
dataset=Dataset(
        dataType=PpgSegment5Min,
        location=env_config['raw_data_folder'],
        project='heart-failure',
        branch=env_config['project_profile'],
        name='raw_ppg_test',
        chunkCount=16,
        repoData=None
    )

Dummy repodata was used, data lineage disabled


In [43]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
def denoising(data):
    MA_WINDOW_SIZE = 3
    SAMPLE_RATE = 64.0
    AVG_WINDOW_SIZE = 2
    LOWER_BOUND = 0
    UPPER_BOUND = 1
    OVERLAP = 0.8
    data = hp.preprocessing.scale_data(
        data, lower=LOWER_BOUND, upper=UPPER_BOUND)

    # Baseline Wander Removal with Wavelet Transform
    # data = baseline_remove(data)

    # Baseline Wander Removal with Notch Filter
    # data = hp.remove_baseline_wander(
    #     data, SAMPLE_RATE, cutoff=0.005)

    # Moving average
    data = window_scaling(
        data, SAMPLE_RATE, windowsize=MA_WINDOW_SIZE, lower=0, upper=1000)

    data = hp.filter_signal(
        data, cutoff=[0.7, 3.5], sample_rate=SAMPLE_RATE, order=3, filtertype='bandpass')

    # calculate bpm and other data
    working_data, measures = hp.process(data, sample_rate=SAMPLE_RATE, windowsize=AVG_WINDOW_SIZE, report_time=False,
                                        calc_freq=True, freq_method='fft', welch_wsize=240, freq_square=False,
                                        interp_clipping=True, clipping_scale=True, interp_threshold=1020,
                                        hampel_correct=False, bpmmin=40, bpmmax=180, reject_segmentwise=False,
                                        high_precision=True, high_precision_fs=128.0, breathing_method='welch',
                                        clean_rr=True, clean_rr_method='z-score', measures=None, working_data=None)

    return data, working_data, measures

def window_scaling(data, sample_rate, windowsize=2.5, lower=0, upper=1024):
    window_dimension = int(windowsize * sample_rate)

    data_start = 0
    data_end = window_dimension

    output = np.empty(len(data))

    while data_end <= len(data):
        sliced = data[data_start:data_end]
        sliced = np.power(sliced, 2)
        scaled = scale_data(sliced, lower, upper)

        output[data_start:data_end] = scaled
        data_start += window_dimension
        data_end += window_dimension

    return np.array(output[0:data_start])

def scale_data(data, lower=0, upper=1024):
    rng = np.max(data) - np.min(data)
    minimum = np.min(data)
    data = (upper - lower) * ((data - minimum) / rng) + lower
    return data

In [48]:
denoise_error_count = 0
error_data = []
with dataset.open('r') as datasetReader:
    for data in datasetReader:
        test = (data)
        try:
            hr, working_data, measures = denoising(test.ppg)
            # hr, working_data, measures = denoising_BL_Notch_WS_BP(sample)
        except Exception as e:
            print("denoiseing error: ", e)
            denoise_error_count+= 1
            error_data.append(test)
denoise_error_count


denoiseing error:  invalid number of data points (0) specified


1

## example

In [5]:
dataset=Dataset(
    dataType=OtherExample,
    location=env_config['raw_data_folder'],
    project='heart-failure',
    branch=env_config['project_profile'],
    name='raw_ppg',
    chunkCount=16,
    repoData=None
)

Dummy repodata was used, data lineage disabled


In [40]:
dataset.delete()

In [6]:
with dataset.open('w') as datasetWriter:
    for k in range(100):
        datasetWriter.append(OtherExample(
            classId=k,
            name=str(k),
            values=[
                ExampleClass(classId=k, value=0.1, name=str(k), creation=datetime.now()),
                ExampleClass(classId=k, value=0.1, name=str(k), creation=datetime.now())
            ],
            dates=[date.today(), date.today()]
        ))

In [9]:
with dataset.open('r') as datasetReader:
    for data in datasetReader:
        print(data)


<class 'src.data.dataset.DatasetReader'>
OtherExample(classId=9, name=9, values=[ExampleClass(classId=9, value=0.1, name=9, creation=2022-07-24 20:33:06.238326), ExampleClass(classId=9, value=0.1, name=9, creation=2022-07-24 20:33:06.238327)], dates=[datetime.date(2022, 7, 24), datetime.date(2022, 7, 24)])
OtherExample(classId=17, name=17, values=[ExampleClass(classId=17, value=0.1, name=17, creation=2022-07-24 20:33:06.238431), ExampleClass(classId=17, value=0.1, name=17, creation=2022-07-24 20:33:06.238431)], dates=[datetime.date(2022, 7, 24), datetime.date(2022, 7, 24)])
OtherExample(classId=28, name=28, values=[ExampleClass(classId=28, value=0.1, name=28, creation=2022-07-24 20:33:06.238663), ExampleClass(classId=28, value=0.1, name=28, creation=2022-07-24 20:33:06.238663)], dates=[datetime.date(2022, 7, 24), datetime.date(2022, 7, 24)])
OtherExample(classId=43, name=43, values=[ExampleClass(classId=43, value=0.1, name=43, creation=2022-07-24 20:33:06.238856), ExampleClass(classId=